# Atividade 3 - Seleção de Modelos com Teste-T

Nome: Juan Felipe Da Silva Rangel

O objetivo desta atividade é explorar o dataset 'mini_mnist.csv', que representa imagens de dígitos de 0-9 manuscritos (contendo 785 colunas), e analisar o desempenho dos classifcadores SVM e KNN utilizando validação cruzada em um único nível. Como já sabemos, este dataset contém uma grande dimensionalidade, e por esse motivo, o PCA será utilizado para reduzir a dimensionalidade.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

%matplotlib inline

In [11]:
# Carrega o dataset
df = pd.read_csv('mini_mnist.csv')
df.head()
# df.shape

255

In [13]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import itertools

In [14]:
# Pego os valores das classes
y = df['digito'].values
X = df.drop('digito', axis=1)

In [15]:
X

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,pixel_774,pixel_775,pixel_776,pixel_777,pixel_778,pixel_779,pixel_780,pixel_781,pixel_782,pixel_783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# PCA
Como foi observado nas células acima, o dataset apresenta 785 colunas, e com o objetivo de diminuir a sua dimensionalidade apenas para duas, será realizado o PCA. Em seguida um scatterplot foi realizado com o objetivo de representar o espaço e verificação de como os diferentes valores da classe de dígitos estão distribuídos.

In [18]:
# Declaro o PCA
pca = PCA(n_components=2)
# X_pca = X

# Calculo e diminuo a dimensionalidade
pca.fit(X)
X_pca = pca.transform(X)

X_pca.shape

# # Crio novamente o dataframe baseado nos valores obtidos
X_pca = pd.DataFrame(X_pca, columns=['PC 0', 'PC 1'])

# Plot de um scatterplot após o PCA
sns.scatterplot(data=X_pca, x='PC 0', y='PC 1', hue=y)
X_pca = X_pca.values

TypeError: 'tuple' object is not callable

In [6]:
from joblib import Parallel, delayed

# Validação Cruzada

Realizo a validação cruzada de um nível para os classificadores KNN e SVM, fazendo uma busca exaustiva para otimizar os hiperparâmetros de ambos classificadores. Também fiquei curioso qual seria a diferença entre a acurácia com e sem realizar o PCA, por esse motivo a validação cruzada foi feita duas vezes.

In [7]:
def selecionar_melhor_k(ks, X_treino, X_val, y_treino, y_val):
    acuracias_val = []

    for k in ks:
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_treino, y_treino)
        pred = knn.predict(X_val)
        acuracias_val.append(accuracy_score(y_val, pred))
        
    melhor_val = max(acuracias_val)
    melhor_k = ks[np.argmax(acuracias_val)]        
    knn = KNeighborsClassifier(n_neighbors=melhor_k)
    knn.fit(np.vstack((X_treino, X_val)), [*y_treino, *y_val])
    
    return knn, melhor_k, melhor_val

In [8]:
# Realiza o treinamento para cada hiperparâmetro
def treina_svm(C, gamma, X_treino, X_val, y_treino, y_val):
    svm = SVC(C= C, gamma= gamma)
    svm.fit(X_treino, y_treino)
    pred = svm.predict(X_val)
    return accuracy_score(y_val, pred)

def selecionar_melhor_svm(Cs, gamma,X_treino:np.ndarray, X_val:np.ndarray, y_treino:np.ndarray, y_val:np.ndarray, n_jobs=4):
    acuracias_val_svm = []
    # cria todas as combinações possíveis entre os Cs e os gammas
    hiperparametros = list(itertools.product(Cs, gamma))
    
    # Treina os modelos utilizando todas as combinações possíveis
    acuracias_val_svm = Parallel(n_jobs= n_jobs)(delayed(treina_svm)
            (c, g, X_treino, X_val, y_treino, y_val) for c, g in hiperparametros)


    melhor_acuracia = max(acuracias_val_svm)
    melhor_combinacao = hiperparametros[np.argmax(acuracias_val_svm)]
    
    svm = SVC(C= melhor_combinacao[0], gamma=melhor_combinacao[1])
    svm.fit(np.vstack((X_treino, X_val)), [*y_treino, *y_val])


    return svm, melhor_combinacao[0], melhor_combinacao[1], melhor_acuracia

def treinando_svm_knn(X_svm_knn):
    # realiza validação cruzada em um nível
    k_vias = 10
    skf = StratifiedKFold(n_splits=k_vias, shuffle=True, random_state=1)
    Cs=[1, 10, 100, 1000]
    gammas=['scale', 'auto', 2e-2, 2e-3, 2e-4]
    i = 0

    pgb = tqdm(total=k_vias, desc='Folds avaliados')
    print("Começando treino em cada fold")

    lista_acuracia_knn = []
    lista_acuracia_svm = []

    for idx_treino, idx_teste in skf.split(X_svm_knn, y):
        print("------------")
        print("Fold %d" % i)
        print("------------")
        # Separando cada fold em treino e teste
        X_treino = X_svm_knn[idx_treino]
        y_treino = y[idx_treino]

        X_teste = X_svm_knn[idx_teste]
        y_teste = y[idx_teste]

        # Separando o conjunto de treino em validação e treino
        X_treino, X_val, y_treino, y_val = train_test_split(X_treino, y_treino, test_size=0.2, stratify= y_treino, random_state=1)

        # Normaliza cada partição
        ss = StandardScaler()
        ss.fit(X_treino)
        X_treino = ss.transform(X_treino)
        X_val = ss.transform(X_val)
        X_teste = ss.transform(X_teste)

        # Otimizando hiperparâmetro do knn
        knn, _, _ = selecionar_melhor_k(range(1,30,2), X_treino, X_val, y_treino, y_val)
        svm, _, _, _ = selecionar_melhor_svm(Cs, gammas,X_treino, X_val, y_treino, y_val)

        # Apartir do melhor modelo treinado do knn e do svm realizo a predição
        pred_svm = svm.predict(X_teste)
        pred_knn = knn.predict(X_teste)

        # Utilizo a acurácia como medida para avaliar o classificador
        acuracia_knn = accuracy_score(y_teste, pred_knn)
        acuracia_svm = accuracy_score(y_teste, pred_svm)

        lista_acuracia_knn.append(acuracia_knn)
        lista_acuracia_svm.append(acuracia_svm)
        print("Acurácia SVM para o fold %d: %.2f" % (i, acuracia_svm))
        print("Acurácia KNN para o fold %d: %.2f" % (i, acuracia_knn))

        # 
        pgb.update(1)
        i += 1

    pgb.close()
    print("Desvio Padrão KNN: %.2f +- %.2f Acurácia Max KNN: %.2f Acurácia Min KNN: %.2f" % (np.mean(lista_acuracia_knn), np.std(lista_acuracia_knn), max(lista_acuracia_knn), min(lista_acuracia_knn)))
    print("Desvio Padrão SVM: %.2f +- %.2f Acurácia Max SVM: %.2f Acurácia Min SVM: %.2f" % (np.mean(lista_acuracia_svm), np.std(lista_acuracia_svm), max(lista_acuracia_svm), min(lista_acuracia_svm)))
    return lista_acuracia_knn, lista_acuracia_svm

In [16]:
print("Acurácia utilizando os dados obtidos depois do PCA")
knn_acuracias, svm_acuracias = treinando_svm_knn(X_pca)

Acurácia utilizando os dados obtidos depois do PCA


Folds avaliados:   0%|          | 0/10 [00:00<?, ?it/s]

Começando treino em cada fold
------------
Fold 0
------------
Acurácia SVM para o fold 0: 0.42
Acurácia KNN para o fold 0: 0.41
------------
Fold 1
------------
Acurácia SVM para o fold 1: 0.43
Acurácia KNN para o fold 1: 0.39
------------
Fold 2
------------
Acurácia SVM para o fold 2: 0.45
Acurácia KNN para o fold 2: 0.39
------------
Fold 3
------------
Acurácia SVM para o fold 3: 0.41
Acurácia KNN para o fold 3: 0.36
------------
Fold 4
------------
Acurácia SVM para o fold 4: 0.45
Acurácia KNN para o fold 4: 0.43
------------
Fold 5
------------
Acurácia SVM para o fold 5: 0.41
Acurácia KNN para o fold 5: 0.39
------------
Fold 6
------------
Acurácia SVM para o fold 6: 0.43
Acurácia KNN para o fold 6: 0.44
------------
Fold 7
------------
Acurácia SVM para o fold 7: 0.47
Acurácia KNN para o fold 7: 0.47
------------
Fold 8
------------
Acurácia SVM para o fold 8: 0.39
Acurácia KNN para o fold 8: 0.42
------------
Fold 9
------------
Acurácia SVM para o fold 9: 0.39
Acurácia KNN 

In [17]:
print("Acurácia utilizando os dados sem realizar o PCA")
knn_acuracias_sem_pca, svm_acuraciassem_pca = treinando_svm_knn(X.values)

Acurácia utilizando os dados sem realizar o PCA


Folds avaliados:   0%|          | 0/10 [00:00<?, ?it/s]

Começando treino em cada fold
------------
Fold 0
------------
Acurácia SVM para o fold 0: 0.87
Acurácia KNN para o fold 0: 0.80
------------
Fold 1
------------
Acurácia SVM para o fold 1: 0.90
Acurácia KNN para o fold 1: 0.89
------------
Fold 2
------------
Acurácia SVM para o fold 2: 0.86
Acurácia KNN para o fold 2: 0.83
------------
Fold 3
------------
Acurácia SVM para o fold 3: 0.87
Acurácia KNN para o fold 3: 0.83
------------
Fold 4
------------
Acurácia SVM para o fold 4: 0.91
Acurácia KNN para o fold 4: 0.81
------------
Fold 5
------------
Acurácia SVM para o fold 5: 0.84
Acurácia KNN para o fold 5: 0.82
------------
Fold 6
------------
Acurácia SVM para o fold 6: 0.88
Acurácia KNN para o fold 6: 0.83
------------
Fold 7
------------
Acurácia SVM para o fold 7: 0.85
Acurácia KNN para o fold 7: 0.85
------------
Fold 8
------------
Acurácia SVM para o fold 8: 0.88
Acurácia KNN para o fold 8: 0.87
------------
Fold 9
------------
Acurácia SVM para o fold 9: 0.88
Acurácia KNN 

# Aplicando Teste-T

É possível perceber acima que as acurácias médias do KNN e do SVM são diferentes, onde pode se esperar, para dados desconhecidos, uma acurácia média entre 0.81 e 0.87 para o classificador SVM e uma acurácia entre 0.86 e 0.90. Podemos notar que há uma pequena sobreposição entre as acurárias, mas a pergunta que deve ser realmente feita é, embora as acurácias sejam diferentes, elas são significativamente significantes, ou seja, é possível negar a hipótese nula que diz que não há diferença entre as duas distribuições.
Será utilizado um alfa de 0.05, que é o valor mínimo que será aceito para rejeitar a hipótese nula, ou seja, será possível dizer com 95% de certeza que a diferença entre as duas distribuições é significativa.

In [18]:
# Calcula os valores de p-value
from scipy.stats import ttest_ind_from_stats

In [19]:
def calcula_media_std(acuracia_classificador):
    return np.mean(acuracia_classificador), np.std(acuracia_classificador)

In [20]:
media_knn, std_knn = calcula_media_std(knn_acuracias)
media_svm, std_svm = calcula_media_std(svm_acuracias)

print("Média KNN: %.2f Desvio Padrão KNN: %.2f" % (media_knn, std_knn))
print("Média KNN: %.2f Desvio Padrão KNN: %.2f" % (media_svm, std_svm))

Média KNN: 0.41 Desvio Padrão KNN: 0.03
Média KNN: 0.43 Desvio Padrão KNN: 0.02


In [21]:
# Calcula p-value
_, pvalor = ttest_ind_from_stats(media_svm, std_svm, len(svm_acuracias), media_knn, std_knn, len(knn_acuracias))

if(pvalor<=0.05):
    print("É possível rejeitar a hipótese nula")
else:
    print("Não é possível rejeitar a hipótese nula")

print("P-Value tem valor igual à: %.4f" % pvalor)


Não é possível rejeitar a hipótese nula
P-Value tem valor igual à: 0.1821


Como não foi possível rejeitar a hipótese nula, não podemos afirmar que a diferença nos resultados da média é significativa entre os dois classificadores, já que o valor de p-value foi maior do que 0.05